<a href="https://colab.research.google.com/github/daawonn/dl_project/blob/master/7_%EC%BC%80%EB%9D%BC%EC%8A%A4%EB%A5%BC%EC%9D%B4%EC%9A%A9%ED%95%9C_CNN_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 절차
- 시퀀스 -> add() -> compile() -> fit() -> 평가

In [0]:
import keras
from keras import models, layers, backend
from keras.datasets import mnist
%tensorflow_version 1.x

# 데이터 확보, 준비

In [96]:
len(mnist.load_data())

2

In [0]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()
# type(train),len(train)

In [98]:
x_train.shape ,y_train.shape
# y_train, y_test는 벡터화처리 해야한다

((60000, 28, 28), (60000,))

In [0]:
# 이미지 크기
img_rows, img_cols = x_train.shape[1:]

In [100]:
# 채널을 설명하는 값에 의해 데이터에 채널값을 앞 혹은 뒤에 추가하여
# Dememsion(차원)을 확장한다 -> 텐서플로우에서는 보이지 않았던 부분
backend.image_data_format()

'channels_last'

In [102]:
x_train[0][0] #dtype=uint8 -> float32로 변환

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]], dtype=uint8)

In [0]:
# 배열의 형변환
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [0]:
# 데이터의 정규화 ~/256 (총 분류갯수) , ~/255 (최대값으로 정규화) 
x_train /= 255
x_test  /= 255

In [105]:
# 레이블 데이터 -> 벡터화
y_train[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=uint8)

In [106]:
# 10 :y_train.unique : 0~9 까지
y_train = keras.utils.to_categorical(y_train, 10) 
y_test = keras.utils.to_categorical(y_test, 10) 
y_train.shape, y_train[0]

((60000, 10), array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32))

# CNN 레이어 설계
- 시퀀스 -> add() -> compile() -> fit() -> 평가

In [0]:
# 모델준비(시퀀스)
model = models.Sequential()

In [0]:
# 합성곱층 추가 : 채널수 증가, 특징이 추출
# filters : 32  => 출력채널수 
# kernel_size : 가중치(W) shape
# strides = (1,1) => 커널을 1칸씩 이동
# padding : 보정, 잘라진 이미지의 크기를 보정하기 ㅜ이해서 행렬을 늘려서 shape을 맞춤
# activation : 활성화 함수
# input_shape : 입력데이터의 shape
model.add( layers.Conv2D( 32,
                         kernel_size =(3,3),
                         strides = (1,1),
                         padding ='valid',
                         activation = 'relu',
                         input_shape = input_shape
                         ) )

In [0]:
# 풀링층 추가 : 이미지사이즈 감소, 특징이 강화
model.add( layers.MaxPool2D( pool_size = (2,2),
                            strides = None , # 기본값 적용
                            ) )

In [0]:
# 과적합방지 : 데이터에 적응되는것을 방지, 학습을 방해(부분적)
# 0.25 : 설정값
model.add( layers.Dropout( 0.25 ) )

In [0]:
# 합성곱층 추가 : 채널수 증가, 특징이 추출
model.add( layers.Conv2D( 64,(3,3),activation='relu') )

In [0]:
# 풀링층 추가 : 이미지사이즈 감소, 특징이 강화
model.add( layers.MaxPool2D( pool_size = (2,2) ) )

In [0]:
# 과적합방지 : 데이터에 적응되는것을 방지, 학습을 방해(부분적)
model.add( layers.Dropout( 0.25 ) )

In [0]:
# 데이터를 펴준다 Flatten 처리
model.add( layers.Flatten() )

In [0]:
# 전결합층 : 데이터를 1차로 펴준다. Flatten
# layers.Dense() => 일반 계층 1개
# 7 x 7 x 64 => Flatten => 128
model.add( layers.Dense( 128 ,activation='relu') )

In [0]:
# 과적합방지 : 데이터에 적응되는것을 방지, 학습을 방해(부분적)
model.add( layers.Dropout( 0.5 ) )

In [0]:
# 출력층 : 데이터를 10개(레이블의 분류 개수만큼)로 수렴
model.add( layers.Dense( 10 ,activation='softmax') )

# 학습 및 평가

In [0]:
# 크로스엔트로피, 경사하강법
# 손실값을 줄이기 위해 조금씩 이동시키면서 학습을 수행한다
model.compile( optimizer= 'rmsprop' ,
              loss = keras.losses.categorical_crossentropy ,
              metrics = ['accuracy']
              )

In [0]:
# 총 학습의 횟수 = 훈련세대수 
epochs     = 10
# 1회 학습시 사용하는 데이터의 양 
batch_size = 128

In [124]:
model.fit( x_train, y_train,
          batch_size = batch_size,
          epochs = epochs, 
          validation_split =0.25 
          )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 45000 samples, validate on 15000 samples
Epoch 1/10





45000/45000 [==============================] - 41s 918us/step - loss: 1.9885 - acc: 0.4997 - val_loss: 1.7366 - val_acc: 0.5853
Epoch 2/10
45000/45000 [==============================] - 40s 881us/step - loss: 1.5589 - acc: 0.5792 - val_loss: 1.3711 - val_acc: 0.5850
Epoch 3/10
45000/45000 [==============================] - 40s 888us/step - loss: 1.2446 - acc: 0.5844 - val_loss: 1.0997 - val_acc: 0.5896
Epoch 4/10
45000/45000 [==============================] - 40s 887us/step - loss: 1.0252 - acc: 0.5884 - val_loss: 0.9240 - val_acc: 0.5893
Epoch 5/10
45000/45000 [==============================] - 40s 879us/step - loss: 0.8864 - acc: 0.5920 - val_loss: 0.8184 - val_acc: 0.5897
Epoch 6/10
45000/45000 [==============================] - 40s 881us/step - loss: 0.8044 - acc: 0.5930 - val_loss: 0.7572 - val_acc: 0.5949
Epoch 7/10
450